In [1]:
import logging
%load_ext autoreload
%autoreload 2

In [260]:
import pandas as pd
import numpy as np
import os
from typing import Union, List
from datetime import datetime, time, timedelta
from dateutil.relativedelta import relativedelta
from utils import calc_date2maturity,calc_time2maturity


from py_vollib_vectorized.implied_volatility import vectorized_implied_volatility

root = '/Users/hongyifan/Desktop/work/internship/citic_futures/20231226bid-ask-spread/'
data_root = root + 'data/'
option_data_root = root + 'data/Option/'
future_data_root = root + 'data/Future/'
res_root = root + 'res/'

trading_dates=np.array(pd.read_csv(data_root + f'trading_dates_cn_2000-01-01_2024-12-31.csv')['trading_dates'].tolist(),dtype=str)

# 获取数据


In [ ]:
import rqdatac

rq_username = "license"
rq_password = "ccU7Yf9dPikhFXtVXsLQZWM0qiY4qxByfselEs2ZWpd01alOGjGpY2R9hXF1_5dG8PKoaJU99ONuKD56i7ySeKpc0lL5G2AP67B5YuhqEH6pUJ-0DSVHVDWLiS4oFqYGmXSNvHU9i0HIScwhQfbwH9gaMwSIkwMStfEvYxV-zxk=a4YImMRv749w2h25tcyJ691JLtZw7QHlqIEcOycx-rZIUQATTdwgF-jcgDxBU5RN09nHRCE2m77u6EDXzStBNPqXzV3gofzOlhXfp9I5o-6bfOPCqUlIFYNt7O3BkppTdRYo2faD3fCI7oc6EKSp64khcP0BaSnNecN1Gm4tJKA="


def init_rqdatac_account():
    """
    该函数用于初始化米筐账户连接。
    """
    try:
        rqdatac.init(
            username=rq_username,
            password=rq_password,
            addr=("rqdatad-pro.ricequant.com", 16011),
            use_pool=True,
            max_pool_size=1,
            auto_load_plugins=False,
        )
        print("米筐账户连接初始化成功")
        quota = rqdatac.user.get_quota()
        print('bytes remain', quota['bytes_limit'] - quota['bytes_used'])
    except Exception as e:
        print(f"米筐账户连接初始化失败，错误信息：{e}")
        raise e


init_rqdatac_account()

In [129]:
# 历史和未来交易日信息
start = '2000-01-01'
end = '2024-12-31'
market = 'cn'
trading_dates = rqdatac.get_trading_dates(start, end, market=market)
pd.Series(trading_dates).astype(str).rename('trading_dates').to_csv(data_root + f'trading_dates_{market}_{start}_{end}.csv',index=False)

# 预处理

## 基础函数

In [142]:

# def calc_date2maturity(current:Union[str,datetime,np.datetime64],expiry:Union[str,datetime,np.datetime64],trading_dates:Union[np.ndarray,List[str]]=None):
#     """用于计算current距离expiry在中国交易日历中的天数
#     
#     Parameters
#     ----------
#     current : str,
#         当前日期
#     expiry : str,
#         到期日
#     trading_dates : np.ndarray,
#         预先存成csv的数据。`trading_dates_cn_2000-01-01_2024-12-31.csv`
#         
#     Returns
#     -------
# 
#     """
#     if isinstance(current,(datetime,np.datetime64)) or isinstance(expiry,(datetime,np.datetime64)): raise NotImplementedError()
#     if trading_dates is None: trading_dates=np.array(pd.read_csv(data_root + f'trading_dates_{market}_{start}_{end}.csv')['trading_dates'].tolist(),dtype=str)
#     if (not current in trading_dates) or (not expiry in trading_dates): raise  ValueError('输入日期为非交易日')
#     
#     idx_curr=np.argwhere(trading_dates==current)[0][0]
#     idx_exp=np.argwhere(trading_dates==expiry)[0][0]
#     return idx_exp-idx_curr

# calc_date2maturity(current='2000-01-11',expiry='2000-01-11',trading_dates=trading_dates)

0

## 期权数据

In [192]:
# 获取所有日期
dates_list=os.listdir(option_data_root)
dates_list.remove('.DS_Store')
dates_list = sorted(dates_list)
dates_list=pd.to_datetime(dates_list,format='%Y-%m-%d').astype(str).tolist()

In [268]:
# 遍历所有csv计算iv
option_info_df = pd.read_csv(data_root + 'etf_options_basic_info.csv')
option_info_df['code'] = option_info_df['code'].apply(lambda x: x[:-5])
option_info_df = option_info_df.set_index('code').sort_index()

exclude = ['510050.csv', '510300.csv', '510500.csv']
files=[]
for date_ in dates_list:
    print(option_data_root+date_.replace('-','')+'/')
    for r,_,f in os.walk(option_data_root+date_.replace('-','')+'/'):
        for ff in f:
            # 去掉中证1000股指期权
            if not ff.startswith('.') and ff not in exclude and not ff.startswith('MO'):
                # print(r,ff)
                files.append(ff)
    #             option_df = pd.read_csv(os.path.join(r, ff))
    #             break
    break
sorted(list(set(files)))

/Users/hongyifan/Desktop/work/internship/citic_futures/20231226bid-ask-spread/data/Option/20230801/


['10005105.csv',
 '10005106.csv',
 '10005107.csv',
 '10005108.csv',
 '10005109.csv',
 '10005110.csv',
 '10005111.csv',
 '10005112.csv',
 '10005113.csv',
 '10005114.csv',
 '10005115.csv',
 '10005116.csv',
 '10005117.csv',
 '10005118.csv',
 '10005119.csv',
 '10005120.csv',
 '10005121.csv',
 '10005122.csv',
 '10005123.csv',
 '10005124.csv',
 '10005125.csv',
 '10005126.csv',
 '10005127.csv',
 '10005128.csv',
 '10005129.csv',
 '10005130.csv',
 '10005131.csv',
 '10005132.csv',
 '10005133.csv',
 '10005134.csv',
 '10005135.csv',
 '10005136.csv',
 '10005137.csv',
 '10005138.csv',
 '10005139.csv',
 '10005140.csv',
 '10005141.csv',
 '10005142.csv',
 '10005143.csv',
 '10005144.csv',
 '10005145.csv',
 '10005146.csv',
 '10005147.csv',
 '10005148.csv',
 '10005149.csv',
 '10005150.csv',
 '10005151.csv',
 '10005152.csv',
 '10005153.csv',
 '10005154.csv',
 '10005155.csv',
 '10005156.csv',
 '10005157.csv',
 '10005158.csv',
 '10005159.csv',
 '10005160.csv',
 '10005167.csv',
 '10005168.csv',
 '10005169.csv

In [212]:
option_symbol=files[0][:-4]
date_,option_symbol

('2023-08-01', '10005735')

In [217]:
expire_date=option_info_df.loc[option_symbol,'expire_date']
expire_date

'2023-08-23'

In [222]:
option_price_df=pd.read_csv(option_data_root+date_.replace('-','')+'/'+f'{option_symbol}.csv')
option_price_df=option_price_df.loc[~option_price_df['auctionLabel']]
option_price_df

,exchangeTime,serverTime,instrumentID,status,source,last,bid1,bid2,bid3,bid4,...,bsize4,bsize5,asize1,asize2,asize3,asize4,asize5,volume,turnover,auctionLabel
12,2023-08-01 09:25:00.175,2023-08-01 09:25:00.097088,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
13,2023-08-01 09:25:17.400,2023-08-01 09:25:17.321283,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
14,2023-08-01 09:25:47.399,2023-08-01 09:25:47.318026,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
15,2023-08-01 09:26:17.399,2023-08-01 09:26:17.319109,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
16,2023-08-01 09:26:47.400,2023-08-01 09:26:47.317038,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4191,2023-08-01 15:02:47.400,2023-08-01 15:02:46.888405,10005735,E,NaN,0.0009,0.0008,0.0006,0.0004,0.0001,...,21,0,21,30,13,15,17,930,8951.0,False
4192,2023-08-01 15:03:17.399,2023-08-01 15:03:16.884883,10005735,E,NaN,0.0009,0.0008,0.0006,0.0004,0.0001,...,21,0,21,30,13,15,17,930,8951.0,False
4193,2023-08-01 15:03:47.400,2023-08-01 15:03:46.886231,10005735,E,NaN,0.0009,0.0008,0.0006,0.0004,0.0001,...,21,0,21,30,13,15,17,930,8951.0,False
4194,2023-08-01 15:04:17.399,2023-08-01 15:04:16.885815,10005735,E,NaN,0.0009,0.0008,0.0006,0.0004,0.0001,...,21,0,21,30,13,15,17,930,8951.0,False


In [249]:


# def calc_time2maturity(current: Union[pd.DatetimeIndex,pd.Series], expiry: Union[pd.DatetimeIndex],days2maturity=None,annualized=240):
#     """
#     
#     Parameters
#     ----------
#     current : 
#     expiry : 
#     days2maturity : 
#     annualized : Optional, int,
#         一年总共的交易日，用于将t2m年化
# 
#     Returns
#     -------
# 
#     """
#     if not isinstance(current,(list)):
#         # 现阶段仅支持current和expiry都在同一天的计算
#         assert days2maturity is not None
#         # 计算time to maturity
#         if days2maturity is not None:
#             if len(set(expiry.date).difference(set(current.date)))>=1:
#                 raise ValueError(f'input has different dates {set(expiry.date).difference(set(current.date))}, conflict with param `days2maturity`')
#         elif days2maturity is None:
#             days2maturity = expiry.date - current.date  # 计算有多少天?
#             
#         # 计算time to maturity
#         seconds2maturity = (expiry-current).total_seconds() +(expiry-current).microseconds/1000000 # 计算距离maturity还有多少秒
#         seconds2maturity=np.array(seconds2maturity)
#         is_morning = seconds2maturity >2*60*60
#         seconds2maturity[is_morning]=seconds2maturity[is_morning]-1.5*60*60
#         
#         time2maturity = seconds2maturity / (4 * 60 * 60) + days2maturity
#         if annualized is not None:
#             time2maturity /= annualized
#         time2maturity = np.where(time2maturity < 1e-3, 1e-3,
#                                  time2maturity)  # For small TTEs, use a small value (1e-3).即最后一小时的TTM都视为1h
#         return time2maturity
#     else:
#         raise NotImplementedError()

In [263]:
from backtest.preprocessors.preprocess import LobTimePreprocessor
ltp=LobTimePreprocessor()

expire_date=option_info_df.loc[option_symbol,'expire_date']
days2maturity = calc_date2maturity(current=date_,expiry=expire_date,trading_dates=trading_dates)  # 计算有多少天

option_price_df=pd.read_csv(option_data_root+date_.replace('-','')+'/'+f'{option_symbol}.csv',parse_dates=True,index_col='serverTime')
option_price_df=option_price_df.loc[~option_price_df['auctionLabel']]
option_price_df=ltp.del_untrade_time(option_price_df)

expiry=pd.to_datetime([date_+' 15:00:00.000000']*len(option_price_df))
current=pd.DatetimeIndex(pd.to_datetime(option_price_df['serverTime']))
time2maturity=calc_time2maturity(current=current,expiry=expiry,days2maturity=days2maturity,annualized=240)

AttributeError: module 'backtest.config' has no attribute 'strip_time'

In [254]:
option_price_df

,exchangeTime,serverTime,instrumentID,status,source,last,bid1,bid2,bid3,bid4,...,bsize4,bsize5,asize1,asize2,asize3,asize4,asize5,volume,turnover,auctionLabel
12,2023-08-01 09:25:00.175,2023-08-01 09:25:00.097088,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
13,2023-08-01 09:25:17.400,2023-08-01 09:25:17.321283,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
14,2023-08-01 09:25:47.399,2023-08-01 09:25:47.318026,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
15,2023-08-01 09:26:17.399,2023-08-01 09:26:17.319109,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
16,2023-08-01 09:26:47.400,2023-08-01 09:26:47.317038,10005735,C,NaN,0.0000,0.0008,0.0007,0.0005,0.0001,...,22,0,40,72,135,18,18,0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4191,2023-08-01 15:02:47.400,2023-08-01 15:02:46.888405,10005735,E,NaN,0.0009,0.0008,0.0006,0.0004,0.0001,...,21,0,21,30,13,15,17,930,8951.0,False
4192,2023-08-01 15:03:17.399,2023-08-01 15:03:16.884883,10005735,E,NaN,0.0009,0.0008,0.0006,0.0004,0.0001,...,21,0,21,30,13,15,17,930,8951.0,False
4193,2023-08-01 15:03:47.400,2023-08-01 15:03:46.886231,10005735,E,NaN,0.0009,0.0008,0.0006,0.0004,0.0001,...,21,0,21,30,13,15,17,930,8951.0,False
4194,2023-08-01 15:04:17.399,2023-08-01 15:04:16.885815,10005735,E,NaN,0.0009,0.0008,0.0006,0.0004,0.0001,...,21,0,21,30,13,15,17,930,8951.0,False


In [252]:
time2maturity*240

array([17.02088929, 17.0196776 , 17.01759472, ..., 15.98425191,
       15.98216864, 15.98008583])

In [253]:
days2maturity

16

In [270]:
option_info_df.loc['10005100':'10005130']

,trading_code,name,contract_type,exchange_code,currency_id,underlying_symbol,underlying_name,underlying_exchange,underlying_type,exercise_price,...,list_price,high_limit,low_limit,expire_date,last_trade_date,exercise_date,delivery_date,is_adjust,delist_date,delist_reason
code,,,,,,,,,,,,,,,,,,,,,
10005100,510300P2302M04600,300ETF沽2月4600,PO,XSHG,CNY,510300.XSHG,华泰柏瑞沪深300ETF,XSHG,ETF,4.60,...,0.4420,0.8578,0.0262,2023-02-22,2023-02-22,2023-02-22,2023-02-23,0,2023-02-23,合约到期
10005101,510300C2303M04600,300ETF购3月4600,CO,XSHG,CNY,510300.XSHG,华泰柏瑞沪深300ETF,XSHG,ETF,4.60,...,0.0226,0.3942,0.0001,2023-03-22,2023-03-22,2023-03-22,2023-03-23,0,2023-03-23,合约到期
10005102,510300P2303M04600,300ETF沽3月4600,PO,XSHG,CNY,510300.XSHG,华泰柏瑞沪深300ETF,XSHG,ETF,4.60,...,0.4420,0.8578,0.0262,2023-03-22,2023-03-22,2023-03-22,2023-03-23,0,2023-03-23,合约到期
10005103,510300C2306M04600,300ETF购6月4600,CO,XSHG,CNY,510300.XSHG,华泰柏瑞沪深300ETF,XSHG,ETF,4.60,...,0.0747,0.4463,0.0001,2023-06-28,2023-06-28,2023-06-28,2023-06-29,0,2023-06-29,合约到期
10005104,510300P2306M04600,300ETF沽6月4600,PO,XSHG,CNY,510300.XSHG,华泰柏瑞沪深300ETF,XSHG,ETF,4.60,...,0.4586,0.8744,0.0428,2023-06-28,2023-06-28,2023-06-28,2023-06-29,0,2023-06-29,合约到期
10005105,510050C2309M02650,50ETF购9月2650,CO,XSHG,CNY,510050.XSHG,50ETF,XSHG,ETF,2.65,...,0.3160,0.6009,0.0311,2023-09-27,2023-09-27,2023-09-27,2023-09-28,0,2023-09-28,合约到期
10005106,510050C2309M02700,50ETF购9月2700,CO,XSHG,CNY,510050.XSHG,50ETF,XSHG,ETF,2.70,...,0.2793,0.5642,0.0001,2023-09-27,2023-09-27,2023-09-27,2023-09-28,0,2023-09-28,合约到期
10005107,510050C2309M02750,50ETF购9月2750,CO,XSHG,CNY,510050.XSHG,50ETF,XSHG,ETF,2.75,...,0.2506,0.5355,0.0001,2023-09-27,2023-09-27,2023-09-27,2023-09-28,0,2023-09-28,合约到期
10005108,510050C2309M02800,50ETF购9月2800,CO,XSHG,CNY,510050.XSHG,50ETF,XSHG,ETF,2.80,...,0.2228,0.5077,0.0001,2023-09-27,2023-09-27,2023-09-27,2023-09-28,0,2023-09-28,合约到期
